In [ ]:
# default_exp vr_parser

In [ ]:
#hide_input
from pivotpy.utils import nav_links 
nav_links(1)

<style>a{text-decoration: none !important;color:lightkblue;font-weight:bold;}
                a:focus,a:active,a:hover{color:hotpink !important;}</style>
> [&nbsp;`▶` Index&nbsp;](https://massgh.github.io/pivotpy/)  
> [&nbsp;`▶` XmlElementTree●&nbsp;](https://massgh.github.io/pivotpy/XmlElementTree)  
> [&nbsp;`▶` StaticPlots&nbsp;](https://massgh.github.io/pivotpy/StaticPlots)  
> [&nbsp;`▶` InteractivePlots&nbsp;](https://massgh.github.io/pivotpy/InteractivePlots)  
> [&nbsp;`▶` Utilities&nbsp;](https://massgh.github.io/pivotpy/Utilities)  
> [&nbsp;`▶` StructureIO&nbsp;](https://massgh.github.io/pivotpy/StructureIO)  
> [&nbsp;`▶` Widgets&nbsp;](https://massgh.github.io/pivotpy/Widgets)  
> [&nbsp;`▶` MainAPI&nbsp;](https://massgh.github.io/pivotpy/MainAPI)  
> [&nbsp;`▶` SpinProjectedSurfaces&nbsp;](https://massgh.github.io/pivotpy/SpinProjectedSurfaces)  


# Xml Parser
> This parser contains functions to extract data from vasprun.xml. All functions in xml parser can work without arguments if working directory contains `vasprun.xml`.

- Almost every object in this module returns a `Dict2Data` object with attributes accessible via dot notation. This object can by transformed to a dictionary by `to_dict()` method on the object.

In [ ]:
#export
import re
import os
from itertools import islice, chain, product
from collections import namedtuple
from contextlib import suppress

import numpy as np
from importlib.machinery import SourceFileLoader
import textwrap
import xml.etree.ElementTree as ET
# Inside packages import to work both with package and jupyter notebook.
try:
    from pivotpy import utils as gu, serializer as serializer
    from pivotpy.sio import read_ticks
except:
    import pivotpy.utils as gu
    import pivotpy.serializer as serializer
    import pivotpy.sio.read_ticks as read_ticks

In [ ]:
#hide_input
# To run notebook smoothly, not for module export
from nbdev.showdoc import show_doc
from pivotpy.vr_parser import load_export, islice2array

In [ ]:
#export
def dict2tuple(name,d):
    """Converts a dictionary (nested as well) to namedtuple, accessible via index and dot notation as well as by unpacking.
    - **Parameters**
        - name: Name of the tuple.
        - d   : Dictionary, nested works as well.
    """
    return namedtuple(name,d.keys())(
           *(dict2tuple(k.upper(),v) if isinstance(v,dict) else v for k,v in d.items())
           )

In [ ]:
show_doc(serializer.Dict2Data)
show_doc(serializer.Dict2Data.to_dict)
show_doc(serializer.Dict2Data.to_json)
show_doc(serializer.Dict2Data.to_pickle)
show_doc(serializer.Dict2Data.to_tuple)

<h2 id="Dict2Data" class="doc_header"><code>class</code> <code>Dict2Data</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L19" class="source_link" style="float:right">[source]</a></h2>

> <code>Dict2Data</code>(**`d`**)

- Returns a Data object with dictionary keys as attributes of Data accessible by dot notation or by key. Once an attribute is created, it can not be changed from outside.
- **Parmeters**
    - dict : Python dictionary (nested as well) containing any python data types.
- **Methods**
    - to_dict  : Converts a Data object to dictionary if it could be made a dictionary, otherwise throws relevant error.
    - to_json  : Converts to json str or save to file if `outfil` given. Accepts `indent` as parameter.
    - to_pickle: Converts to bytes str or save to file if `outfile` given.
    - to_tuple : Converts to a named tuple.
- **Example**
    > x = Dict2Data({'A':1,'B':{'C':2}})
    > x
    > Data(
    >     A = 1
    >     B = Data(
    >         C = 2
    >         )
    >     )
    > x.B.to_dict()
    > {'C': 2}

<h4 id="Dict2Data.to_dict" class="doc_header"><code>Dict2Data.to_dict</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L52" class="source_link" style="float:right">[source]</a></h4>

> <code>Dict2Data.to_dict</code>()

Converts a `Dict2Data` object (root or nested level) to a dictionary.
        

<h4 id="Dict2Data.to_json" class="doc_header"><code>Dict2Data.to_json</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L62" class="source_link" style="float:right">[source]</a></h4>

> <code>Dict2Data.to_json</code>(**`outfile`**=*`None`*, **`indent`**=*`1`*)

Dumps a `Dict2Data` object (root or nested level) to json.
- **Parameters**
    - outfile : Default is None and returns string. If given, writes to file.
    - indent  : Json indent. Default is 1.

<h4 id="Dict2Data.to_pickle" class="doc_header"><code>Dict2Data.to_pickle</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L70" class="source_link" style="float:right">[source]</a></h4>

> <code>Dict2Data.to_pickle</code>(**`outfile`**=*`None`*)

Dumps a `Dict2Data` object (root or nested level) to pickle.
- **Parameters**
    - outfile : Default is None and returns string. If given, writes to file.

<h4 id="Dict2Data.to_tuple" class="doc_header"><code>Dict2Data.to_tuple</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L77" class="source_link" style="float:right">[source]</a></h4>

> <code>Dict2Data.to_tuple</code>()

Creates a namedtuple.

In [ ]:
x = serializer.Dict2Data({'A':1,'B':2})
print('Dict: ',x.to_dict())
print('JSON: ',x.to_json())
print('Pickle: ',x.to_pickle())
print('Tuple: ',x.to_tuple())
x['A']


Dict:  {'A': 1, 'B': 2}
JSON:  {
 "A": 1,
 "B": 2
}
Pickle:  b'\x80\x03}q\x00(X\x01\x00\x00\x00Aq\x01K\x01X\x01\x00\x00\x00Bq\x02K\x02u.'
Tuple:  Data(A=1, B=2)


1

## Parser Functions

In [ ]:
#export
def read_asxml(path = None):
    """
    - Reads a big vasprun.xml file into memory once and then apply commands. If current folder contains `vasprun.xml` file, it automatically picks it.

    - **Parameters**
        - path : Path/To/vasprun.xml

    - **Returns**
        - xml_data : Xml object to use in other functions
    """
    path = path or './vasprun.xml'
    if not os.path.isfile(path):
        raise FileNotFoundError("File: '{}'' does not exist!".format(path))
        
    elif 'vasprun.xml' not in path:
        raise Exception("File name should end with 'vasprun.xml'")
        
    fsize = gu.get_file_size(path)
    value = float(fsize.split()[0])
    print_str = """
    Memory Consumption Warning!
    ---------------------------
    File: {} is large ({}). It may consume a lot of memory (generally 3 times the file size).
        An alternative way is to parse vasprun.xml is by using `Vasp2Visual` module in Powershell by command `pivotpy.load_export('path/to/vasprun.xml'), which runs underlying powershell functions to load data whith efficient memory managment. It works on Windows/Linux/MacOS if you have powershell core and Vasp2Visual installed on it.
    """.format(path,fsize)
    if 'MB' in fsize and value > 200:
        print(gu.color.y(textwrap.dedent(print_str)))
    elif 'GB' in fsize and value > 1:
        print(gu.color.y(textwrap.dedent(print_str)))
    
    nt = namedtuple('VasprunXML',['path','root']) # Safe and full of info
    return nt(os.path.abspath(path), ET.parse(path).getroot()) # THis is xml_data for other functions

In [ ]:
#export
def xml2dict(xmlnode_or_filepath):
    """Convert xml node or xml file content to dictionary. All output text is in string format, so further processing is required to convert into data types/split etc.
    - The only paramenter `xmlnode_or_filepath` is either a path to an xml file or an `xml.etree.ElementTree.Element` object.
    - Each node has `tag,text,attr,nodes` attributes. Every text element can be accessed via
    `xml2dict()['nodes'][index]['nodes'][index]...` tree which makes it simple.
    """
    if isinstance(xmlnode_or_filepath,str):
        node = read_asxml(xmlnode_or_filepath)
    else:
        node = xmlnode_or_filepath

    text = node.text.strip() if node.text else ''
    nodes = [xml2dict(child) for child in list(node)]
    return {'tag': node.tag,'text': text, 'attr':node.attrib, 'nodes': nodes}

In [ ]:
#export
def exclude_kpts(xml_data):
    """
    - Returns number of kpoints to exclude used from IBZKPT.
    - **Parameters**
        - xml_data : From `read_asxml` function
    - **Returns**
        - int      : Number of kpoints to exclude.
    """
    for kpts in xml_data.root.iter('varray'):
        if(kpts.attrib=={'name': 'weights'}):
            weights=[float(arr.text.strip()) for arr in kpts.iter('v')]
    exclude=[]
    [exclude.append(item) for item in weights if item!=weights[-1]];
    skipk=len(exclude) #that much to skip
    return skipk

In [ ]:
#export
def get_ispin(xml_data):
    """
    - Returns value of ISPIN.
    - **Parameters**
        - xml_data : From `read_asxml` function
    - **Returns**
        - int      : Value of ISPIN.
    """
    for item in xml_data.root.iter('i'):
        if(item.attrib=={'type': 'int', 'name': 'ISPIN'}):
            return int(item.text)

In [ ]:
#export
def get_kpoints_info(other_path = './vasprun.xml'):
    "Read KPOINTS file header and Cartesian/Reciprocal information based on other_path like vasprun.xml."
    base_dir = os.path.split(os.path.abspath(other_path))[0]
    path = os.path.join(base_dir,'KPOINTS')
    if not os.path.isfile(path):
        raise FileNotFoundError(f"File {path!r} not found. KPOINTS file should be in same directory as of {other_path!r}.")
    line = islice2array(path,start=2,nlines=1,exclude=None,raw=True).strip()
    cart = True if line[0] in 'cCkK' else False 
    header = islice2array(path,start=0,nlines=1,exclude=None,raw=True).strip()
    return serializer.Dict2Data({'cartesian':cart,'header':header})
    

def get_summary(xml_data):
    """
    - Returns overview of system parameters.
    - **Parameters**
        - xml_data : From `read_asxml` function
    - **Returns**
        - Data     : pivotpy.Dict2Data with attibutes accessible via dot notation.
    """
    for i_car in xml_data.root.iter('incar'):
        incar={car.attrib['name']:car.text.strip() for car in i_car}
    n_ions=[int(atom.text) for atom in xml_data.root.iter('atoms')][0]
    type_ions=[int(atom_types.text) for atom_types in xml_data.root.iter('types')][0]
    elem=[info[0].text.strip() for info in xml_data.root.iter('rc')]
    elem_name=[]; #collect IONS names
    [elem_name.append(item) for item in elem[:-type_ions] if item not in elem_name]
    elem_index=[0]; #start index
    [elem_index.append((int(entry)+elem_index[-1])) for entry in elem[-type_ions:]];
    ISPIN=get_ispin(xml_data=xml_data)
    NELECT = int([i.text.strip().split('.')[0] for i in xml_data.root.iter('i') if i.attrib['name']=='NELECT'][0])
    # Fields
    try:
        for pro in xml_data.root.iter('partial'):
            dos_fields=[field.text.strip() for field in pro.iter('field')]
            dos_fields = [field for field in dos_fields if 'energy' not in field]
    except:
        dos_fields = []
    for i in xml_data.root.iter('i'): #efermi for condition required.
        if(i.attrib=={'name': 'efermi'}):
            efermi=float(i.text)
    #Writing information to a dictionary
    info_dic={'SYSTEM':incar['SYSTEM'],'NION':n_ions,'NELECT':NELECT,'TypeION':type_ions,
              'ElemName':elem_name,'ElemIndex':elem_index,'E_Fermi': efermi,'ISPIN':ISPIN,
              'fields':dos_fields,'incar':incar,'kpts_info':get_kpoints_info(other_path = xml_data.path)}
    return serializer.Dict2Data(info_dic)

In [ ]:
import pivotpy.vr_parser as vp
xml_data = read_asxml(path= '../vasprun.xml')
get_summary(xml_data=xml_data).to_tuple()

Data(SYSTEM='unknown system', NION=3, NELECT=14, TypeION=2, ElemName=['Mg', 'B'], ElemIndex=[0, 1, 3], E_Fermi=5.09505239, ISPIN=2, fields=['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'dx2'], incar=INCAR(SYSTEM='unknown system', PREC='accurate', ALGO='Fast', ISPIN='2', ICHARG='1', NELM='100', IBRION='2', EDIFF='0.00000100', NSW='99', ISIF='3', ENCUT='520.00000000', MAGMOM='0.60000000      0.60000000      0.60000000', LREAL='Auto', ISMEAR='0', SIGMA='0.05000000', LWAVE='F', LORBIT='F'), kpts_info=KPTS_INFO(cartesian=False, header="Automatically generated using PivotPy with HSK-INDS = [0, 30, 60,-1], LABELS = ['Γ','M','K', 'Γ'], SEG-INDS = []"))

In [ ]:
xml_data

VasprunXML(path='E:\\Research\\vasprun.xml', root=<Element 'modeling' at 0x0000025EC459B3B8>)

In [ ]:
#export
def join_ksegments(kpath,kseg_inds=[]):
    """Joins a broken kpath's next segment to previous. `kseg_inds` should be list of first index of next segment"""
    path_list = np.array(kpath)
    if kseg_inds:
        for ind in kseg_inds:
            path_list[ind:] -= path_list[ind] - path_list[ind-1]
    return list(path_list)

def get_kpts(xml_data, skipk = 0,kseg_inds=[]):
    r"""Returns kpoints and calculated kpath.

    **Parameters**
    - xml_data: From `read_asxml` function.
    - skipk : (int) Number of initil kpoints to skip.
    - kseg_inds : (list) List of indices of kpoints where path is broken.

    **Returns**
    - Data : pivotpy.Dict2Data with attibutes `kpath` and `kpoints`.
    """
    for kpts in xml_data.root.iter('varray'):
        if(kpts.attrib=={'name': 'kpointlist'}):
            kpoints=[[float(item) for item in arr.text.split()] for arr in kpts.iter('v')]
    kpoints=np.array(kpoints[skipk:])
    #KPath solved.
    kpath=[0];pts=kpoints
    [kpath.append(np.round(np.sqrt(np.sum((pt1-pt2)**2))+kpath[-1],6)) for pt1,pt2 in zip(pts[:-1],pts[1:])]
    # If broken path, then join points.
    kpath = join_ksegments(kpath,kseg_inds)
    return serializer.Dict2Data({'NKPTS':len(kpoints),'kpoints':kpoints,'kpath':kpath})

In [ ]:
get_kpts(xml_data=xml_data,skipk=10)

Data(
    NKPTS = 30
    kpoints = <ndarray:shape=(30, 3)>
    kpath = <list:len=30>
)

In [ ]:
#export
def get_tdos(xml_data,spin_set=1,elim=[]):
    """
    - Returns total dos for a spin_set (default 1) and energy limit. If spin-polarized calculations, gives SpinUp and SpinDown keys as well.
    - **Parameters**
        - xml_data : From `read_asxml` function
        - spin_set : int, default is 1 to pick up total spin.
        - elim     : List [min,max] of energy, default empty.
    - **Returns**
        - Data     : pivotpy.Dict2Data with attibutes E_Fermi, ISPIN,tdos.
    """
    tdos=[]; #assign for safely exit if wrong spin set entered.
    ISPIN = get_ispin(xml_data=xml_data)
    for neighbor in xml_data.root.iter('dos'):
        for item in neighbor[1].iter('set'):
            if(ISPIN==1 and spin_set==1):
                if(item.attrib=={'comment': 'spin 1'}):
                    tdos=np.array([[float(entry) for entry in arr.text.split()] for arr in item])
            if(ISPIN==2 and spin_set==1):
                if(item.attrib=={'comment': 'spin 1'}):
                    tdos_1=np.array([[float(entry) for entry in arr.text.split()] for arr in item])
                if(item.attrib=={'comment': 'spin 2'}):
                    tdos_2=np.array([[float(entry) for entry in arr.text.split()] for arr in item])
                    tdos = {'SpinUp':tdos_1,'SpinDown':tdos_2}
            if(spin_set!=1): #can get any
                if(item.attrib=={'comment': 'spin {}'.format(spin_set)}):
                    tdos=np.array([[float(entry) for entry in arr.text.split()] for arr in item])
    for i in xml_data.root.iter('i'): #efermi for condition required.
        if(i.attrib=={'name': 'efermi'}):
            efermi=float(i.text)
    dos_dic= {'E_Fermi':efermi,'ISPIN':ISPIN,'tdos':tdos}
    #Filtering in energy range.
    if elim: #check if elim not empty
        if(ISPIN==1 and spin_set==1):
            up_ind=np.max(np.where(tdos[:,0]-efermi<=np.max(elim)))+1
            lo_ind=np.min(np.where(tdos[:,0]-efermi>=np.min(elim)))
            tdos=tdos[lo_ind:up_ind,:]
        if(ISPIN==2 and spin_set==1):
            up_ind=np.max(np.where(tdos['SpinUp'][:,0]-efermi<=np.max(elim)))+1
            lo_ind=np.min(np.where(tdos['SpinUp'][:,0]-efermi>=np.min(elim)))
            tdos = {'SpinUp':tdos_1[lo_ind:up_ind,:],'SpinDown':tdos_2[lo_ind:up_ind,:]}
        if(spin_set!=1):
            up_ind=np.max(np.where(tdos[:,0]-efermi<=np.max(elim)))+1
            lo_ind=np.min(np.where(tdos[:,0]-efermi>=np.min(elim)))
            tdos=tdos[lo_ind:up_ind,:]
        dos_dic= {'E_Fermi':efermi,'ISPIN':ISPIN,'grid_range':range(lo_ind,up_ind),'tdos':tdos}
    return serializer.Dict2Data(dos_dic)

In [ ]:
get_tdos(xml_data=xml_data,spin_set=1,elim=[])

Data(
    E_Fermi = 5.09505239
    ISPIN = 2
    tdos = Data(
        SpinUp = <ndarray:shape=(301, 3)>
        SpinDown = <ndarray:shape=(301, 3)>
    )
)

In [ ]:
#export
def get_evals(xml_data,skipk=None,elim=[]):
    """
    - Returns eigenvalues as numpy array. If spin-polarized calculations, gives SpinUp and SpinDown keys as well.
    - **Parameters**
        - xml_data : From `read_asxml` function
        - skipk    : Number of initil kpoints to skip.
        - elim     : List [min,max] of energy, default empty.
    - **Returns**
        - Data     : pivotpy.Dict2Data with attibutes evals and related parameters.
    """
    evals=[]; #assign for safely exit if wrong spin set entered.
    ISPIN=get_ispin(xml_data=xml_data)
    if skipk!=None:
        skipk=skipk
    else:
        skipk=exclude_kpts(xml_data=xml_data) #that much to skip by default
    for neighbor in xml_data.root.iter('eigenvalues'):
        for item in neighbor[0].iter('set'):
            if(ISPIN==1):
                if(item.attrib=={'comment': 'spin 1'}):
                    evals=np.array([[float(th.text.split()[0]) for th in thing] for thing in item])[skipk:]
                    NBANDS=len(evals[0])
            if(ISPIN==2):
                if(item.attrib=={'comment': 'spin 1'}):
                    eval_1=np.array([[float(th.text.split()[0]) for th in thing] for thing in item])[skipk:]
                if(item.attrib=={'comment': 'spin 2'}):
                    eval_2=np.array([[float(th.text.split()[0]) for th in thing] for thing in item])[skipk:]
                    evals={'SpinUp':eval_1,'SpinDown':eval_2}
                    NBANDS=len(eval_1[0])

    for i in xml_data.root.iter('i'): #efermi for condition required.
        if(i.attrib=={'name': 'efermi'}):
            efermi=float(i.text)
    evals_dic={'E_Fermi':efermi,'ISPIN':ISPIN,'NBANDS':NBANDS,'evals':evals,'indices': range(NBANDS)}
    if elim: #check if elim not empty
        if(ISPIN==1):
            up_ind=np.max(np.where(evals[:,:]-efermi<=np.max(elim))[1])+1
            lo_ind=np.min(np.where(evals[:,:]-efermi>=np.min(elim))[1])
            evals=evals[:,lo_ind:up_ind]
        if(ISPIN==2):
            up_ind=np.max(np.where(eval_1[:,:]-efermi<=np.max(elim))[1])+1
            lo_ind=np.min(np.where(eval_1[:,:]-efermi>=np.min(elim))[1])
            evals={'SpinUp':eval_1[:,lo_ind:up_ind],'SpinDown':eval_2[:,lo_ind:up_ind]}
        NBANDS = int(up_ind - lo_ind) #update Bands
        evals_dic['NBANDS'] = NBANDS
        evals_dic['indices'] = range(lo_ind,up_ind)
        evals_dic['evals'] = evals
    return serializer.Dict2Data(evals_dic)

In [ ]:
get_evals(xml_data=xml_data,skipk=10,elim=[-5,5])

Data(
    E_Fermi = 5.09505239
    ISPIN = 2
    NBANDS = 5
    evals = Data(
        SpinUp = <ndarray:shape=(30, 5)>
        SpinDown = <ndarray:shape=(30, 5)>
    )
    indices = range(4, 9)
)

In [ ]:
#export
def get_bands_pro_set(xml_data,
                      spin_set=1,
                      skipk=0,
                      bands_range=None,
                      set_path=None):
    """
    - Returns bands projection of a spin_set(default 1). If spin-polarized calculations, gives SpinUp and SpinDown keys as well.
    - **Parameters**
        - xml_data    : From `read_asxml` function
        - skipk       : Number of initil kpoints to skip (Default 0).
        - spin_set    : Spin set to get, default is 1.
        - bands_range : If elim used in `get_evals`,that will return bands_range to use here. Note that range(0,2) will give 2 bands 0,1 but tuple (0,2) will give 3 bands 0,1,2.
        - set_path     : path/to/_set[1,2,3,4].txt, works if `split_vasprun` is used before.
    - **Returns**
        - Data     : pivotpy.Dict2Data with attibutes of bands projections and related parameters.
    """
    if bands_range != None:
        check_list = list(bands_range)
        if check_list==[]:
            raise ValueError("No bands prjections found in given energy range.")
    # Try to read _set.txt first. instance check is important.
    if isinstance(set_path,str) and os.path.isfile(set_path):
        _header = islice2array(set_path,nlines=1,raw=True,exclude=None)
        _shape = [int(v) for v in _header.split('=')[1].strip().split(',')]
        NKPTS, NBANDS, NIONS, NORBS = _shape
        if NORBS == 3:
            fields = ['s','p','d']
        elif NORBS == 9:
            fields = ['s','py','pz','px','dxy','dyz','dz2','dxz','x2-y2']
        else:
            fields = [str(i) for i in range(NORBS)] #s,p,d in indices.
        COUNT = NIONS*NBANDS*(NKPTS-skipk)*NORBS
        start = NBANDS*NIONS*skipk
        nlines = None # Read till end.
        if bands_range:
            _b_r = list(bands_range)
            # First line is comment but it is taken out by exclude in islice2array.
            start = [[NIONS*NBANDS*k + NIONS*b for b in _b_r] for k in range(skipk,NKPTS)]
            start = [s for ss in start for s in ss] #flatten
            nlines = NIONS # 1 band has nions
            NBANDS = _b_r[-1]-_b_r[0]+1 # upadte after start

        NKPTS = NKPTS-skipk # Update after start, and bands_range.
        COUNT = NIONS*NBANDS*NKPTS*NORBS
        data = islice2array(set_path,start=start,nlines=nlines,count=COUNT)
        data = data.reshape((NKPTS,NBANDS,NIONS,NORBS)).transpose([2,0,1,3])
        return serializer.Dict2Data({'labels':fields,'pros':data})

    #Collect Projection fields
    fields=[];
    for pro in xml_data.root.iter('projected'):
        for arr in pro.iter('field'):
            if('eig' not in arr.text and 'occ' not in arr.text):
                fields.append(arr.text.strip())
    NORBS = len(fields)
    #Get NIONS for reshaping data
    NIONS=[int(atom.text) for atom in xml_data.root.iter('atoms')][0]

    for spin in xml_data.root.iter('set'):
        if spin.attrib=={'comment': 'spin{}'.format(spin_set)}:
            k_sets = [kp for kp in spin.iter('set') if 'kpoint' in kp.attrib['comment']]
    k_sets = k_sets[skipk:]
    NKPTS = len(k_sets)
    band_sets = []
    for k_s in k_sets:
        b_set = [b for b in k_s.iter('set') if 'band' in b.attrib['comment']]
        if bands_range == None:
            band_sets.extend(b_set)
        else:
            b_r = list(bands_range)
            band_sets.extend(b_set[b_r[0]:b_r[-1]+1])
    NBANDS = int(len(band_sets)/len(k_sets))
    try:
        # Error prone solution but 5 times fater than list comprehension.
        bands_pro = (float(t) for band in band_sets for l in band.iter('r') for t in l.text.split())
        COUNT = NKPTS*NBANDS*NORBS*NIONS # Must be counted for performance.
        data = np.fromiter(bands_pro,dtype=float,count=COUNT)
    except:
        # Alternate slow solution
        print("Error using `np.fromiter`.\nFalling back to (slow) list comprehension...",end=' ')
        bands_pro = (l.text for band in band_sets for l in band.iter('r'))
        bands_pro = [[float(t) for t in text.split()] for text in bands_pro]
        data = np.array(bands_pro)
        del bands_pro # Release memory
        print("Done.")

    data = data.reshape((NKPTS,NBANDS,NIONS,NORBS)).transpose((2,0,1,3))
    return serializer.Dict2Data({'labels':fields,'pros':data})

In [ ]:
get_bands_pro_set(xml_data,skipk=0,spin_set=1,bands_range=range(0, 1))

Data(
    labels = ['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'dx2']
    pros = <ndarray:shape=(3, 40, 1, 9)>
)

In [ ]:
#export
def get_dos_pro_set(xml_data,spin_set=1,dos_range=None):
    """
    - Returns dos projection of a spin_set(default 1) as numpy array. If spin-polarized calculations, gives SpinUp and SpinDown keys as well.
    - **Parameters**
        - xml_data    : From `read_asxml` function
        - spin_set    : Spin set to get, default 1.
        - dos_range   : If elim used in `get_tdos`,that will return dos_range to use here..
    - **Returns**
        - Data     : pivotpy.Dict2Data with attibutes of dos projections and related parameters.
    """
    if dos_range != None:
        check_list = list(dos_range)
        if check_list == []:
            raise ValueError("No DOS prjections found in given energy range.")
    
    n_ions=get_summary(xml_data=xml_data).NION
    for pro in xml_data.root.iter('partial'):
        dos_fields=[field.text.strip()for field in pro.iter('field')]
        #Collecting projections.
        dos_pro=[]; set_pro=[]; #set_pro=[] in case spin set does not exists
        for ion in range(n_ions):
            for node in pro.iter('set'):
                if(node.attrib=={'comment': 'ion {}'.format(ion+1)}):
                    for spin in node.iter('set'):
                        if(spin.attrib=={'comment': 'spin {}'.format(spin_set)}):
                            set_pro=[[float(entry) for entry in r.text.split()] for r in spin.iter('r')]
            dos_pro.append(set_pro)
    if dos_range==None: #full grid computed.
        dos_pro=np.array(dos_pro) #shape(NION,e_grid,pro_fields)
    else:
        dos_range=list(dos_range)
        min_ind=dos_range[0]
        max_ind=dos_range[-1]+1
        dos_pro=np.array(dos_pro)[:,min_ind:max_ind,:]
    final_data=np.array(dos_pro) #shape(NION,e_grid,pro_fields)
    return serializer.Dict2Data({'labels':dos_fields,'pros':final_data})

In [ ]:
#export
def _is_poscar_cartesian(other_path = './vaprun.xml'):
    base_dir = os.path.split(os.path.abspath(other_path))[0]
    _path = os.path.join(base_dir, 'POSCAR')
    if not os.path.isfile(_path):
        raise FileNotFoundError(f"File {_path!r} not found. POSCAR file is required from same directory as of {other_path!r}.")
    lines = islice2array(_path,start=7,nlines=2,exclude=None,raw=True).splitlines()
    lines = [l.strip() for l in lines] # remove whitespace or tabs
    cart =  True if ((lines[0][0] in 'cCkK') or (lines[1][0] in 'cCkK')) else False
    return cart

def get_structure(xml_data):
    """
    - Returns structure's volume,basis,positions and rec-basis.
    - **Parameters**
        - xml_data : From `read_asxml` function.
    - **Returns**
        - Data     : pivotpy.Dict2Data with attibutes volume,basis,positions rec_basis and labels.
    """
    SYSTEM = [i.text for i in xml_data.root.iter('i') if i.attrib['name'] == 'SYSTEM'][0]

    for final in xml_data.root.iter('structure'):
        if(final.attrib=={'name': 'finalpos'}):
            for i in final.iter('i'):
                volume=float(i.text)
            for arr in final.iter('varray'):
                if(arr.attrib=={'name': 'basis'}):
                    basis=[[float(a) for a in v.text.split()] for v in arr.iter('v')]
                if(arr.attrib=={'name': 'rec_basis'}):
                    rec_basis=[[float(a) for a in v.text.split()] for v in arr.iter('v')]
                if(arr.attrib=={'name': 'positions'}):
                    positions=[[float(a) for a in v.text.split()] for v in arr.iter('v')]
    # element labels
    types  = [int(_type.text) for _type in xml_data.root.iter('types')][0]
    elems  = [info[0].text.strip() for info in xml_data.root.iter('rc')]
    _inds  = np.array([int(a) for a in elems[-types:]])
    _nums  = [k + 1 for i in _inds for k in range(i)]
    labels = [f"{e} {i}" for i, e in zip(_nums,elems)]

    INDS = np.cumsum([0,*_inds]).astype(int)
    Names = list(np.unique(elems[:-types]))
    unique_d = {e:range(INDS[i],INDS[i+1]) for i,e in enumerate(Names)}

    st_dic={'SYSTEM':SYSTEM,'volume': volume,'basis': np.array(basis),'rec_basis': np.array(rec_basis),
            'cartesian': _is_poscar_cartesian(other_path=xml_data.path),'positions': np.array(positions),'labels':labels,'unique': unique_d}
    return serializer.PoscarData(st_dic)

In [ ]:
get_structure(xml_data)

PoscarData(
    SYSTEM = unknown system
    volume = 28.80748507
    basis = <ndarray:shape=(3, 3)>
    rec_basis = <ndarray:shape=(3, 3)>
    cartesian = False
    positions = <ndarray:shape=(3, 3)>
    labels = ['Mg 1', 'B 1', 'B 2']
    unique = Data(
        B = range(0, 1)
        Mg = range(1, 3)
    )
    text_plain = unknown system # Exported using pivotpy
  3.08577910000000    
    1.0000000000000000   -0.0000000000000000    0.0000000000000000
   -0.5000000000000000    0.8660254164013231    0.0000000000000000
   -0.0000000000000000   -0.0000000000000000    1.1320880551689523
  B	Mg
  1	2
Direct
  -0.0000000000000000   0.0000000000000000  -0.0000000000000000
   0.3333330000000000   0.6666670000000000   0.5000000000000000
   0.6666670000000000   0.3333330000000000   0.5000000000000000
)

## Quick Export for Bandstructure
A fully comprehensive command that uses all functions and returns data for spin set 1 (set 1 and 2 if spin-polarized calculations) could be constructed for immediate usage. It is `export_vasrun()`.

In [ ]:
#export
def export_vasprun(path        = None,
                   skipk       = None,
                   elim        = [],
                   kseg_inds   = [],
                   shift_kpath = 0,
                   try_pwsh    = True
                   ):
    """
    - Returns a full dictionary of all objects from `vasprun.xml` file. It first try to load the data exported by powershell's `Export-VR(Vasprun)`, which is very fast for large files. It is recommended to export large files in powershell first.
    - **Parameters**
        - path       : Path to `vasprun.xml` file. Default is `'./vasprun.xml'`.
        - skipk      : Default is None. Automatically detects kpoints to skip.
        - elim       : List [min,max] of energy interval. Default is [], covers all bands.
        - kseg_inds : List of indices of kpoints where path is broken.
        - shift_kpath: Default 0. Can be used to merge multiple calculations on single axes side by side.
        - try_pwsh   : Default is True and tries to load data exported by `Vasp2Visual` in Powershell.
    - **Returns**
        - Data : Data accessible via dot notation containing nested Data objects:
            - sys_info  : System Information
            - dim_info  : Contains information about dimensions of returned objects.
            - kpoints   : numpy array of kpoints with excluded IBZKPT points
            - kpath     : 1D numpy array directly accessible for plot.
            - bands     : Data containing bands.
            - tdos      : Data containing total dos.
            - pro_bands : Data containing bands projections.
            - pro_dos   : Data containing dos projections.
            - poscar    : Data containing basis,positions, rec_basis and volume.
    """
    # Try to get files if exported data in PowerShell.
    if try_pwsh:
        req_files = ['Bands.txt','tDOS.txt','pDOS.txt','Projection.txt','SysInfo.py']
        if path and os.path.isfile(path):
            req_files = [os.path.join(
                os.path.dirname(os.path.abspath(path)),f) for f in req_files]
        logic = [os.path.isfile(f) for f in req_files]
        if not False in logic:
            print('Loading from PowerShell Exported Data...')
            return load_export(path=(path if path else './vasprun.xml'))

    # Proceed if not files from PWSH
    if path==None:
        path='./vasprun.xml'
        
    xml_data = read_asxml(path=path)
    
    base_dir = os.path.split(os.path.abspath(path))[0]
    set_paths = [os.path.join(base_dir,"_set{}.txt".format(i)) for i in (1,2)]
    #First exclude unnecessary kpoints. Includes only same weight points
    if skipk!=None:
        skipk=skipk
    else:
        skipk = exclude_kpts(xml_data) #that much to skip by default
    info_dic = get_summary(xml_data) #Reads important information of system.
    #KPOINTS
    kpts = get_kpts(xml_data,skipk=skipk,kseg_inds=kseg_inds)
    #EIGENVALS
    eigenvals = get_evals(xml_data,skipk=skipk,elim=elim)
    #TDOS
    tot_dos = get_tdos(xml_data,spin_set=1,elim=elim)
    #Bands and DOS Projection
    if elim:
        bands_range = eigenvals.indices #indices in range form.
        grid_range=tot_dos.grid_range
    else:
        bands_range=None #projection function will read itself.
        grid_range=None
    if(info_dic.ISPIN==1):
        pro_bands = get_bands_pro_set(xml_data=xml_data,spin_set=1,skipk=skipk,bands_range=bands_range,set_path=set_paths[0])
        pro_dos = get_dos_pro_set(xml_data=xml_data,spin_set=1,dos_range=grid_range)
    if(info_dic.ISPIN==2):
        pro_1 = get_bands_pro_set(xml_data=xml_data,spin_set=1,skipk=skipk,bands_range=bands_range,set_path=set_paths[0])
        pro_2 = get_bands_pro_set(xml_data=xml_data,spin_set=2,skipk=skipk,bands_range=bands_range,set_path=set_paths[1])
        pros={'SpinUp': pro_1.pros,'SpinDown': pro_2.pros}#accessing spins in dictionary after .pro.
        pro_bands={'labels':pro_1.labels,'pros': pros}
        pdos_1 = get_dos_pro_set(xml_data=xml_data,spin_set=1,dos_range=grid_range)
        pdos_2 = get_dos_pro_set(xml_data=xml_data,spin_set=1,dos_range=grid_range)
        pdos={'SpinUp': pdos_1.pros,'SpinDown': pdos_2.pros}#accessing spins in dictionary after .pro.
        pro_dos={'labels':pdos_1.labels,'pros': pdos}

    #Structure
    poscar = get_structure(xml_data = xml_data)
    poscar = {'SYSTEM':info_dic.SYSTEM,**poscar.to_dict()}
    #Dimensions dictionary.
    dim_dic={'kpoints':'(NKPTS,3)','kpath':'(NKPTS,1)','bands':'⇅(NKPTS,NBANDS)','dos':'⇅(grid_size,3)','pro_dos':'⇅(NION,grid_size,en+pro_fields)','pro_bands':'⇅(NION,NKPTS,NBANDS,pro_fields)'}
    #Writing everything to be accessible via dot notation
    kpath=[k+shift_kpath for k in kpts.kpath]  # shift kpath for side by side calculations.
    full_dic={'sys_info':info_dic,'dim_info':dim_dic,'kpoints':kpts.kpoints,'kpath':kpath,'bands':eigenvals,
             'tdos':tot_dos,'pro_bands':pro_bands,'pro_dos':pro_dos,'poscar': poscar}
    return serializer.VasprunData(full_dic)

In [ ]:
export_vasprun(path='E:/Research/graphene_example/ISPIN_1/bands/vasprun.xml',elim=[-1,0],try_pwsh=True)

Loading from PowerShell Exported Data...


VasprunData(
    sys_info = Data(
        SYSTEM = C2
        NION = 2
        NELECT = 8
        TypeION = 1
        ElemName = ['C']
        E_Fermi = -3.3501
        fields = ['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'x2-y2']
        incar = Data(
            SYSTEM = C2
            PREC = high
            ALGO = N
            LSORBIT = T
            NELMIN = 7
            ISMEAR = 0
            SIGMA = 0.10000000
            LORBIT = 11
            GGA = PS
        )
        ElemIndex = [0, 2]
        ISPIN = 1
        kpts_info = Data(
            cartesian = False
            header = Automatically generated using PivotPy with HSK-INDS = [0, 30, 60,-1], LABELS = ['Γ','M','K', 'Γ'], SEG-INDS = []
        )
    )
    dim_info = Data(
        kpoints = (NKPTS,3)
        kpath = (NKPTS,1)
        bands = ⇅(NKPTS,NBANDS)
        dos = ⇅(grid_size,3)
        pro_dos = ⇅(NION,grid_size,en+pro_fields)
        pro_bands = ⇅(NION,NKPTS,NBANDS,pro_fields)
    )
    kpoints = <ndar

In [ ]:
#export
def _validate_evr(path_evr=None,**kwargs):
    "Validates data given for plotting functions. Returns a tuple of (Boolean,data)."
    if type(path_evr) == serializer.VasprunData:
        return path_evr
    
    path_evr = path_evr or './vasprun.xml' # default path.
    
    if isinstance(path_evr,str):
        if os.path.isfile(path_evr):
            # kwargs -> skipk=skipk,elim=elim,kseg_inds=kseg_inds
            return export_vasprun(path=path_evr,**kwargs)
        else:
            raise FileNotFoundError(f'File {path_evr!r} not found!')
    # Other things are not valid.
    raise ValueError('path_evr must be a path string or output of export_vasprun function.')

## Joining Multiple Calculations
- Sometimes one may need to compare two or more bandstructures in same figure, for that reason, it is easy to export two calculations and plot on same axis.
- There is another situation, if you have a large supercell and split calculations into multiple ones, joining that calculations works same way, you will add the last value of first kpath into all values of next kpath and next last to next and so on, by just using `shift_kpath` in `export_vasprun` and plot each export on same axis, this will align bandstructures side by side on same axis.


## Load Exported Vasprun from PowerShell
On Windows, it will work automatically. On Linux/Mac it may require path to powershell executable.

In [ ]:
#export
def load_export(path= './vasprun.xml',
                kseg_inds =[],
                shift_kpath = 0,
                path_to_ps='pwsh',
                skipk = None,
                max_filled = 10,
                max_empty = 10,
                keep_files = True
                ):
    """
    - Returns a full dictionary of all objects from `vasprun.xml` file exported using powershell.
    - **Parameters**
        - path       : Path to `vasprun.xml` file. Default is `'./vasprun.xml'`.
        - skipk      : Default is None. Automatically detects kpoints to skip.
        - path_to_ps : Path to `powershell.exe`. Automatically picks on Windows and Linux if added to PATH.
        - kseg_inds : List of indices of kpoints where path is broken.
        - shift_kpath: Default 0. Can be used to merge multiple calculations side by side.
        - keep_files : Could be use to clean exported text files. Default is True.
        - max_filled : Number of filled bands below and including VBM. Default is 10.
        - max_empty  : Number of empty bands above VBM. Default is 10.
    - **Returns**
        - Data : Data accessible via dot notation containing nested Data objects:
            - sys_info  : System Information
            - dim_info  : Contains information about dimensions of returned objects.
            - kpoints   : numpy array of kpoints with excluded IBZKPT points
            - kpath     : 1D numpy array directly accessible for plot.
            - bands     : Data containing bands.
            - tdos      : Data containing total dos.
            - pro_bands : Data containing bands projections.
            - pro_dos   : Data containing dos projections.
            - poscar    : Data containing basis,positions, rec_basis and volume.
    """
    that_loc, file_name = os.path.split(os.path.abspath(path)) # abspath is important to split.
    with gu.set_dir(that_loc):
        # Goes there and work
        i = 0
        required_files = ['Bands.txt','tDOS.txt','pDOS.txt','Projection.txt','SysInfo.py']
        for _file in required_files:
            if os.path.isfile(_file):
               i = i + 1
        if i < 5:
            if skipk != None:
                gu.ps2std(path_to_ps=path_to_ps,ps_command='Import-Module Vasp2Visual; Export-VR -InputFile {} -MaxFilled {} -MaxEmpty {} -SkipK {}'.format(path,max_filled,max_empty,skipk))
            else:
                gu.ps2std(path_to_ps=path_to_ps,ps_command='Import-Module Vasp2Visual; Export-VR -InputFile {} -MaxFilled {} -MaxEmpty {}'.format(path,max_filled,max_empty))
        
        # get info from same directory
        iscartesian = _is_poscar_cartesian()
        kpoints_info = get_kpoints_info()
    
        # Enable loading SysInfo.py file as source.
        _vars = SourceFileLoader("SysInfo", "./SysInfo.py").load_module()
    
        SYSTEM            = _vars.SYSTEM
        NKPTS             = _vars.NKPTS
        NBANDS            = _vars.NBANDS
        NFILLED           = _vars.NFILLED
        TypeION           = _vars.TypeION
        NION              = _vars.NION
        NELECT            = _vars.NELECT
        nField_Projection = _vars.nField_Projection
        E_Fermi           = _vars.E_Fermi
        ISPIN             = _vars.ISPIN
        ElemIndex         = _vars.ElemIndex
        ElemName          = _vars.ElemName
        poscar            = {'SYSTEM': SYSTEM,
                            'volume':_vars.volume,
                            'basis' : np.array(_vars.basis),
                            'rec_basis': np.array(_vars.rec_basis),
                            'cartesian':iscartesian,
                            'positions': np.array(_vars.positions)
                            }
        fields            = _vars.fields
        incar             = _vars.INCAR
    
        # Elements Labels
        elem_labels = []
        for i, name in enumerate(ElemName):
            for ind in range(ElemIndex[i],ElemIndex[i+1]):
                elem_labels.append(f"{name} {str(ind - ElemIndex[i] + 1)}")
        poscar.update({'labels': elem_labels})
        # Unique Elements Ranges
        unique_d = {}
        for i,e in enumerate(ElemName):
            unique_d.update({e:range(ElemIndex[i],ElemIndex[i+1])})
        poscar.update({'unique': unique_d})

        # Load Data
        bands= np.loadtxt('Bands.txt').reshape((-1,NBANDS+4)) #Must be read in 2D even if one row only.
        start = int(open('Bands.txt').readline().split()[4][1:])
        pro_bands= np.loadtxt('Projection.txt').reshape((-1,NBANDS*nField_Projection))
        pro_dos = np.loadtxt('pDOS.txt')
        dos= np.loadtxt('tDOS.txt')
    
        # Keep or delete only if python generates files (i < 5 case.)
        if(keep_files==False and i==5):
            for file in required_files:
                os.remove(file)
        # Returns back

    # Work now!
    sys_info = {'SYSTEM': SYSTEM,'NION': NION,'NELECT':NELECT,'TypeION': TypeION,'ElemName': ElemName, 
                'E_Fermi': E_Fermi,'fields':fields, 'incar': incar,'ElemIndex': ElemIndex,'ISPIN': ISPIN,
                'kpts_info': kpoints_info}
    dim_info = {'kpoints': '(NKPTS,3)','kpath': '(NKPTS,1)','bands': '⇅(NKPTS,NBANDS)','dos': '⇅(grid_size,3)',
                'pro_dos': '⇅(NION,grid_size,en+pro_fields)','pro_bands': '⇅(NION,NKPTS,NBANDS,pro_fields)'}

    bands_dic,tdos_dic,pdos_dic,pro_dic,kpath={},{},{},{},[]
    if(ISPIN==1):
        kpath   = bands[:,3]
        kpoints = bands[:,:3]
        evals   = bands[:,4:]
        bands_dic = {'E_Fermi': E_Fermi, 'ISPIN': ISPIN, 'NBANDS': NBANDS, 'evals': evals, 'indices': range(start,start+NBANDS)}
        tdos_dic  = {'E_Fermi': E_Fermi, 'ISPIN': ISPIN,'tdos': dos}
        pdos      = pro_dos.reshape(NION,-1,nField_Projection+1)
        pdos_dic  = {'labels': fields,'pros': pdos}
        pros      = pro_bands.reshape(NION,NKPTS,NBANDS,-1)
        pro_dic   = {'labels': fields,'pros': pros}
    if(ISPIN==2):
        # Bands
        kpath   = bands[:NKPTS,3]
        kpoints = bands[:NKPTS,:3]
        SpinUp  = bands[:NKPTS,4:]
        SpinDown= bands[NKPTS:,4:]
        evals   = {'SpinUp':SpinUp,'SpinDown': SpinDown}
        bands_dic = {'E_Fermi': E_Fermi, 'ISPIN': ISPIN, 'NBANDS': NBANDS, 'evals': evals,'indices': range(start,start+NBANDS)}
        # tDOS
        dlen    = int(np.shape(dos)[0]/2)
        SpinUp  = dos[:dlen,:]
        SpinDown= dos[dlen:,:]
        tdos    = {'SpinUp':SpinUp,'SpinDown': SpinDown}
        tdos_dic= {'E_Fermi': E_Fermi, 'ISPIN': ISPIN,'tdos': tdos}

        # pDOS
        plen    = int(np.shape(pro_dos)[0]/2)
        SpinUp  = pro_dos[:plen,:].reshape(NION,-1,nField_Projection+1)
        SpinDown= pro_dos[plen:,:].reshape(NION,-1,nField_Projection+1)
        pdos    = {'SpinUp':SpinUp,'SpinDown': SpinDown}
        pdos_dic= {'labels': fields,'pros': pdos}

        # projections
        pblen  = int(np.shape(pro_bands)[0]/2)
        SpinUp  = pro_bands[:pblen,:].reshape(NION,NKPTS,NBANDS,-1)
        SpinDown= pro_bands[pblen:,:].reshape(NION,NKPTS,NBANDS,-1)
        pros    = {'SpinUp': SpinUp,'SpinDown': SpinDown}
        pro_dic = {'labels': fields,'pros': pros}
    # If broken path, then join points.
    kpath = join_ksegments(kpath,kseg_inds)
    kpath=[k+shift_kpath for k in kpath.copy()] # Shift kpath
    full_dic = {'sys_info': sys_info,'dim_info': dim_info,'kpoints': kpoints,'kpath':kpath,               'bands':bands_dic,'tdos':tdos_dic,'pro_bands': pro_dic ,'pro_dos': pdos_dic,
               'poscar':serializer.PoscarData(poscar)}
    return serializer.VasprunData(full_dic)

This back and forth data transport is required in [pivotpy-dash](https://github.com/massgh/pivotpy-dash) app where data is stored in browser in json format, but needs to by python objects for figures.

## Write Clean data to JSON or Pickle file
Use `pivotpy.serializer.dump` to write output of `export_vasprun` or `load_export` to pickle/json file. Pickle is useful for quick load in python while json is useful to transfer data into any language. 

In [ ]:
show_doc(serializer.dump)
show_doc(serializer.load)

<h4 id="dump" class="doc_header"><code>dump</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L227" class="source_link" style="float:right">[source]</a></h4>

> <code>dump</code>(**`dict_data`**=*`None`*, **`dump_to`**=*`'pickle'`*, **`outfile`**=*`None`*, **`indent`**=*`1`*)

- Dump an [`export_vasprun`](/pivotpy/XmlElementTree.html#export_vasprun) or [`load_export`](/pivotpy/XmlElementTree.html#load_export)'s `Data` object or any dictionary to json or pickle string/file. It convert `Dict2Data` to dictionary before serializing to json/pickle, so json/pickle.loads() of converted Data would be a simple dictionary, pass that to `Dict2Data` to again make accessible via dot notation.
- **Parameters**
    - dict_data: Any dictionary/Dict2Data(or subclass Data) object containg numpy arrays, including [`export_vasprun`](/pivotpy/XmlElementTree.html#export_vasprun) or [`load_export`](/pivotpy/XmlElementTree.html#load_export) output.
    - dump_to  : Defualt is `pickle` or `json`.
    - outfile  : Defualt is None and return string. File name does not require extension.
    - indent   : Defualt is 1. Only works for json.

<h4 id="load" class="doc_header"><code>load</code><a href="https://github.com/massgh/pivotpy/tree/master/pivotpy/serializer.py#L257" class="source_link" style="float:right">[source]</a></h4>

> <code>load</code>(**`file_or_str`**, **`json_to`**=*`Dict2Data`*)

- Loads a json/pickle dumped file or string by auto detecting it.
- **Parameters**
    - file_or_str : Filename of pickl/json or their string.
    - json_to     : If loading json, convert to (Dict2Data by defualt) or subclasses and/or dict. 

In [ ]:
import pivotpy as pp 
evr = pp.Vasprun('../vasprun.xml').data

In [ ]:
s = serializer.dump(evr.poscar,dump_to='pickle')
#print(s)
serializer.load(s)

Data(
    SYSTEM = unknown system
    volume = 28.80748507
    basis = <ndarray:shape=(3, 3)>
    rec_basis = <ndarray:shape=(3, 3)>
    cartesian = False
    positions = <ndarray:shape=(3, 3)>
    labels = ['Mg 1', 'B 1', 'B 2']
    unique = Data(
        B = range(0, 1)
        Mg = range(1, 3)
    )
    text_plain = unknown system # Exported using pivotpy
  3.08577910000000    
    1.0000000000000000   -0.0000000000000000    0.0000000000000000
   -0.5000000000000000    0.8660254164013231    0.0000000000000000
   -0.0000000000000000   -0.0000000000000000    1.1320880551689523
  B	Mg
  1	2
Direct
  -0.0000000000000000   0.0000000000000000  -0.0000000000000000
   0.3333330000000000   0.6666670000000000   0.5000000000000000
   0.6666670000000000   0.3333330000000000   0.5000000000000000
)

## Parse Text Files with Flexibility
- The function `islice2array` is used to read text files which have patterns of text and numbers inline, such as EIGENVAL and PROCAR. With all the options of this function, reading and parsing of such files should take a few lines of code only. It can be used to read txt,csv tsv as well with efficent speed.
- It reads a file without fully loading into memory and you can still access slices of data in the file. That partial data fetching from file is very handy.  

In [ ]:
#export
def islice2array(path_or_islice,dtype=float,delimiter='\s+',
                include=None,exclude='#',raw=False,fix_format = True,
                start=0,nlines=None,count=-1,cols=None,new_shape=None
                ):
    """
    - Reads a sliced array from txt,csv type files and return to array. Also manages if columns lengths are not equal and return 1D array. It is faster than loading  whole file into memory. This single function could be used to parse EIGENVAL, PROCAR, DOCAR and similar files with just a combination of `exclude, include,start,stop,step` arguments.
    - **Parameters**
        - path_or_islice: Path/to/file or `itertools.islice(file_object)`. islice is interesting when you want to read different slices of an opened file and do not want to open it again and again. For reference on how to use it just execute `pivotpy.export_potential??` in a notebook cell or ipython terminal to see how islice is used extensively.
        - dtype: float by default. Data type of output array, it is must have argument.
        - start,nlines: The indices of lines to start reading from and number of lines after start respectively. Only work if `path_or_islice` is a file path. both could be None or int, while start could be a list to read slices from file provided that nlines is int. The spacing between adjacent indices in start should be equal to or greater than nlines as pointer in file do not go back on its own.  These parameters are in output of `slice_data`
        > Note: `start` should count comments if `exclude` is None. You can use `slice_data` function to get a dictionary of `start,nlines, count, cols, new_shape` and unpack in argument instead of thinking too much.
        - count: `np.size(output_array) = nrows x ncols`, if it is known before execution, performance is increased. This parameter is in output of `slice_data`.
        - delimiter:  Default is `\s+`. Could be any kind of delimiter valid in numpy and in the file.
        - cols: List of indices of columns to pick. Useful when reading a file like PROCAR which e.g. has text and numbers inline. This parameter is in output of `slice_data`.
        - include: Default is None and includes everything. String of patterns separated by | to keep, could be a regular expression.
        - exclude: Default is '#' to remove comments. String of patterns separated by | to drop,could be a regular expression.
        - raw    : Default is False, if True, returns list of raw strings. Useful to select `cols`.
        - fix_format: Default is True, it sepearates numbers with poor formatting like 1.000-2.000 to 1.000 2.000 which is useful in PROCAR. Keep it False if want to read string literally.
        - new_shape : Tuple of shape Default is None. Will try to reshape in this shape, if fails fallbacks to 2D or 1D. This parameter is in output of `slice_data`.
    - **Examples**
        > `islice2array('path/to/PROCAR',start=3,include='k-point',cols=[3,4,5])[:2]`
        > array([[ 0.125,  0.125,  0.125],
        >        [ 0.375,  0.125,  0.125]])
        > `islice2array('path/to/EIGENVAL',start=7,exclude='E',cols=[1,2])[:2]`
        > array([[-11.476913,   1.      ],
        >        [  0.283532,   1.      ]])
    > Note: Slicing a dimension to 100% of its data is faster than let say 80% for inner dimensions, so if you have to slice more than 50% of an inner dimension, then just load full data and slice after it.
    """
    if nlines is None and isinstance(start,(list,np.ndarray)):
        print("`nlines = None` with `start = array/list` is useless combination.")
        return np.array([]) # return empty array.
        
    def _fixing(_islice,include=include, exclude=exclude,fix_format=fix_format,nlines=nlines,start=start):
        if include:
            _islice = (l for l in _islice if re.search(include,l))
    
        if exclude:
            _islice = (l for l in _islice if not re.search(exclude,l))
    
        # Make slices here after comment excluding.
        if isinstance(nlines,int) and isinstance(start,(list,np.ndarray)):
            #As islice moves the pointer as it reads, start[1:]-nlines-1
            # This confirms spacing between two indices in start >= nlines
            start = [start[0],*[s2-s1-nlines for s1,s2 in zip(start,start[1:])]]
            _islice = chain(*(islice(_islice,s,s+nlines) for s in start))
        elif isinstance(nlines,int) and isinstance(start,int):
            _islice = islice(_islice,start,start+nlines)
        elif nlines is None and isinstance(start,int):
            _islice = islice(_islice,start,None)
    
        # Negative connected digits to avoid, especially in PROCAR
        if fix_format:
            _islice = (re.sub(r"(\d)-(\d)",r"\1 -\2",l) for l in _islice)
        return _islice

    def _gen(_islice,cols=cols):
        for line in _islice:
            line = line.strip().replace(delimiter,'  ').split()
            if line and cols is not None: # if is must here.
                line = [line[i] for i in cols]
            for chars in line:
                yield dtype(chars)
    
    #Process Now
    if isinstance(path_or_islice,str) and os.path.isfile(path_or_islice):
        with open(path_or_islice,'r') as f:
            _islice = islice(f,0,None) # Read full, Will fix later.
            _islice = _fixing(_islice)
            if raw:
                return ''.join(_islice)
            # Must to consume islice when file is open
            data = np.fromiter(_gen(_islice),dtype=dtype,count=count)
    else:
        _islice = _fixing(path_or_islice)
        if raw:
            return ''.join(_islice)
        data = np.fromiter(_gen(_islice),dtype=dtype,count=count)
    
    if new_shape:
        try: data = data.reshape(new_shape)
        except: pass
    elif cols: #Otherwise single array.
        try: data = data.reshape((-1,len(cols)))
        except: pass
    return data

In [ ]:
#export 
def slice_data(dim_inds,old_shape):
    """
    - Returns a dictionary that can be unpacked in arguments of isclice2array function. This function works only for regular txt/csv/tsv data files which have rectangular data written.
    - **Parameters**
        - dim_inds : List of indices array or range to pick from each dimension. Inner dimensions are more towards right. Last itmes in dim_inds is considered to be columns. If you want to include all values in a dimension, you can put -1 in that dimension. Note that negative indexing does not work in file readig, -1 is s special case to fetch all items.
        - old_shape: Shape of data set including the columns length in right most place.
    - **Example**
        - You have data as 3D arry where third dimension is along column.
        > 0 0
        > 0 2
        > 1 0
        > 1 2
        - To pick [[0,2], [1,2]], you need to give
        > slice_data(dim_inds = [[0,1],[1],-1], old_shape=(2,2,2))
        > {'start': array([1, 3]), 'nlines': 1, 'count': 2}
        - Unpack above dictionary in `islice2array` and you will get output array.
    - Note that dimensions are packed from right to left, like 0,2 is repeating in 2nd column.
    """
    # Columns are treated diffiernetly.
    if dim_inds[-1] == -1:
        cols = None
    else:
        cols = list(dim_inds[-1])

    r_shape = old_shape[:-1]
    dim_inds = dim_inds[:-1]
    for i,ind in enumerate(dim_inds.copy()):
        if ind == -1:
            dim_inds[i] = range(r_shape[i])
    nlines = 1
    #start = [[NIONS*NBANDS*k + NIONS*b for b in _b_r] for k in range(skipk,NKPTS)] #kind of thing.
    _prod_ = product(*dim_inds)
    _mult_ = [np.product(r_shape[i+1:]) for i in range(len(r_shape))]
    _out_ = np.array([np.dot(p,_mult_) for p in _prod_]).astype(int)
    # check if innermost dimensions could be chunked.
    step = 1
    for i in range(-1,-len(dim_inds),-1):
        _inds = np.array(dim_inds[i]) #innermost
        if np.max(_inds[1:] - _inds[:-1]) == 1: # consecutive
            step = len(_inds)
            _out_ = _out_[::step] # Pick first indices
            nlines = step*nlines
            # Now check if all indices picked then make chunks in outer dimensions too.
            if step != r_shape[i]: # Can't make chunk of outer dimension if inner is not 100% picked.
                break # Stop more chunking
    new_shape = [len(inds) for inds in dim_inds] #dim_inds are only in rows.
    new_shape.append(old_shape[-1])
    return {'start':_out_,'nlines':nlines,'count': nlines*len(_out_),'cols':cols,'new_shape':tuple(new_shape)}

In [ ]:
slice_data([list(range(1,7)),-1,-1,range(2)],old_shape=[52,768,64,9])


{'start': array([ 49152,  98304, 147456, 196608, 245760, 294912]),
 'nlines': 49152,
 'count': 294912,
 'cols': [0, 1],
 'new_shape': (6, 768, 64, 9)}

## Process Largs `vasprun.xml` Files
You can split a large vasprun.xml file in a small `_vasprun.xml` file which does not contain projected data, and `_set[1,2,3,4].txt` file(s) which contain projected data of each spin set. These spin set text files can be processed by `islice2array` function efficiently.

In [ ]:
#export
def split_vasprun(path=None):
    """
    - Splits a given vasprun.xml file into a smaller _vasprun.xml file plus _set[1,2,3,4].txt files which contain projected data for each spin set.
    - **Parameters**
        - path: path/to/vasprun.xml file.
    - **Output**
        - _vasprun.xml file with projected data.
        - _set1.txt for projected data of colinear calculation.
        - _set1.txt for spin up data and _set2.txt for spin-polarized case.
        - _set[1,2,3,4].txt for each spin set of non-colinear calculations.
    """
    if not path:
        path = './vasprun.xml'
    if not os.path.isfile(path):
        raise FileNotFoundError("{!r} does not exist!".format(path))
    base_dir = os.path.split(os.path.abspath(path))[0]
    out_file = os.path.join(base_dir,'_vasprun.xml')
    out_sets = [os.path.join(base_dir,'_set{}.txt'.format(i)) for i in range(1,5)]
    # process
    with open(path,'r') as f:
        lines = islice(f,None)
        indices = [i for i,l in enumerate(lines) if re.search('projected|/eigenvalues',l)]
        f.seek(0)
        print("Writing {!r} ...".format(out_file),end=' ')
        with open(out_file,'w') as outf:
            outf.write(''.join(islice(f,0,indices[1])))
            f.seek(0)
            outf.write(''.join(islice(f,indices[-1]+1,None)))
            print('Done')
            
        f.seek(0)
        middle = islice(f,indices[-2]+1,indices[-1]) #projected words excluded
        spin_inds = [i for i,l in enumerate(middle) if re.search('spin',l)][1:] #first useless.
        if len(spin_inds)>1:
            set_length = spin_inds[1]-spin_inds[0] # Must define
        else:
            set_length = indices[-1]-indices[-2] #It is technically more than set length, but fine for 1 set
        f.seek(0) # Must be at zero
        N_sets = len(spin_inds)
        # Let's read shape from out_file as well.
        xml_data = read_asxml(out_file)
        _summary = get_summary(xml_data)
        NIONS  = _summary.NION
        NORBS  = len(_summary.fields)
        NBANDS = get_evals(xml_data).NBANDS
        NKPTS  = get_kpts(xml_data).NKPTS
        del xml_data # free meory now.
        for i in range(N_sets): #Reads every set
            print("Writing {!r} ...".format(out_sets[i]),end=' ')
            start = (indices[-2]+1+spin_inds[0] if i==0 else 0) # pointer is there next time.
            stop_ = start + set_length # Should move up to set length only.
            with open(out_sets[i],'w') as setf:
                setf.write("  # Set: {} Shape: (NKPTS[NBANDS[NIONS]],NORBS) = {},{},{},{}\n".format(i+1,NKPTS,NBANDS,NIONS,NORBS))
                middle = islice(f,start,stop_)
                setf.write(''.join(l.lstrip().replace('/','').replace('<r>','') for l in middle if '</r>' in l))
                print('Done')

# Export Data with Specific Spin Set(s)
This is useful for spin textures, fermi surfaces etc.

In [ ]:
#export
def export_spin_data(path = None, spins = 's', skipk = None, elim = None):
    """
    - Returns Data with selected spin sets. For spin polarized calculations, it returns spin up and down data.
    - **Parameters**
        - path   : Path to `vasprun.xml` file. Default is `'./vasprun.xml'`.
        - skipk  : Default is None. Automatically detects kpoints to skip.
        - elim   : List [min,max] of energy interval. Default is [], covers all bands.
        - spins  : Spin components to include from 'sxyz', e.g. 'sx' will pick <S> and <S_x> if present.
                   Only works if ISPIN == 1, otherwise it will be two sets for spin up and down.
    - **Returns**
        - Data : Data accessible via dot notation containing nested Data objects:
            - sys_info  : System Information
            - dim_info  : Contains information about dimensions of returned objects.
            - kpoints   : numpy array of kpoints with excluded IBZKPT points
            - evals     : Data containing eigenvalues as evals.<e,u,d>.
            - spins     : Data containing bands projections as spins.<u,d,s,x,y,z>.
            - poscar    : Data containing basis,positions, rec_basis and volume.
    """
    if not isinstance(spins,str):
        raise TypeError(f"`spins` must be a string from 'sxyz', got {spins}!")
    
    if False in [comp in 'sxyz' for comp in spins]:
        raise ValueError(f"`spins` must be in 'sxyz', got {spins!r}!")

    xml_data = read_asxml(path = path or './vasprun.xml')
    
    base_dir = os.path.split(os.path.abspath(path or './vasprun.xml'))[0]
    set_paths = [os.path.join(base_dir,"_set{}.txt".format(i)) for i in (1,2,3,4)]
    
    skipk = skipk or exclude_kpts(xml_data=xml_data) #that much to skip by default
    full_dic = {'sys_info':get_summary(xml_data)}
    
    ISPIN = full_dic['sys_info'].ISPIN
    LSORBIT = getattr(full_dic['sys_info'].incar, 'LSORBIT', 'FALSE')
    if 'f' in LSORBIT.lower() and ISPIN == 1:
        for comp in spins:
            if comp in 'xyz':
                raise ValueError(f"LSORBIT = {LSORBIT} does not include spin component {comp!r}!")
    
    full_dic['dim_info'] = {'kpoints':'(NKPTS,3)','evals.<e,u,d>':'⇅(NKPTS,NBANDS)','spins.<u,d,s,x,y,z>':'⇅(NION,NKPTS,NBANDS,pro_fields)'}
    full_dic['kpoints']= get_kpts(xml_data, skipk = skipk).kpoints
    
    bands = get_evals(xml_data, skipk = skipk,elim = elim).to_dict()
    evals = bands['evals']
    bands.update({'u': evals['SpinUp'], 'd': evals['SpinDown']} if ISPIN == 2 else {'e': evals})
    del bands['evals']
    full_dic['evals'] = bands
    
    bands_range = full_dic['bands'].indices if elim else None #indices in range form.
    
    spin_sets = {}   
    if ISPIN == 1:
        for n, s in enumerate('sxyz', start = 1):
            if s in spins:
                spin_sets[s] = get_bands_pro_set(xml_data, spin_set = n, skipk = skipk, bands_range = bands_range, set_path = set_paths[n-1]).pros
        
    if ISPIN == 2:
        print(gu.color.g(f"Found ISPIN = 2, output data got attributes spins.<u,d> instead of spins.<{','.join(spins)}>"))
        pro_1 = get_bands_pro_set(xml_data, spin_set = 1, skipk = skipk, bands_range = bands_range, set_path = set_paths[0])
        pro_2 = get_bands_pro_set(xml_data, spin_set = 2, skipk = skipk, bands_range = bands_range, set_path = set_paths[1])
        spin_sets = {'u': pro_1.pros,'d': pro_2.pros} 
        
    full_dic['spins'] = spin_sets
    full_dic['spins']['labels'] = full_dic['sys_info'].fields
    full_dic['poscar'] = {'SYSTEM':full_dic['sys_info'].SYSTEM,**(get_structure(xml_data).to_dict())}
    return serializer.SpinData(full_dic)

In [ ]:
#hide_input
from pivotpy.utils import nav_links 
nav_links(1)

<style>a{text-decoration: none !important;color:lightkblue;font-weight:bold;}
                a:focus,a:active,a:hover{color:hotpink !important;}</style>
> [&nbsp;`▶` Index&nbsp;](https://massgh.github.io/pivotpy/)  
> [&nbsp;`▶` XmlElementTree●&nbsp;](https://massgh.github.io/pivotpy/XmlElementTree)  
> [&nbsp;`▶` StaticPlots&nbsp;](https://massgh.github.io/pivotpy/StaticPlots)  
> [&nbsp;`▶` InteractivePlots&nbsp;](https://massgh.github.io/pivotpy/InteractivePlots)  
> [&nbsp;`▶` Utilities&nbsp;](https://massgh.github.io/pivotpy/Utilities)  
> [&nbsp;`▶` StructureIO&nbsp;](https://massgh.github.io/pivotpy/StructureIO)  
> [&nbsp;`▶` Widgets&nbsp;](https://massgh.github.io/pivotpy/Widgets)  
> [&nbsp;`▶` MainAPI&nbsp;](https://massgh.github.io/pivotpy/MainAPI)  
> [&nbsp;`▶` SpinProjectedSurfaces&nbsp;](https://massgh.github.io/pivotpy/SpinProjectedSurfaces)  
